# Tekstanalyse 4: Custom sentiment
***
***
Keywords: `SpaCy`, `data frames`, `parts of speech tagging`, `POS`, `sentiment analyse`,

Nye Python-udtryk:  `.getcwd()`, `.glob`, `.pos_`, `.join()`, `afinn.score()`, `Counter()`
***
***
I det følgende skal vi arbejde videre med NLP-pakken `SpaCy`. Vi har allerede kigget på mulighederne for at lave **named entity recognition**, og vi skal nu forsøge at kombinere **Parts of Speech Tagging** og **Sentiment analysis** med henblik på at undersøge, i hvilket omfang de seneste ti års statsministernytårstaler har været overvejende positive eller negative.

Nogle af elementerne vil være repetition af elementer fra tidligere notebooks.

Hvis der er kode sekvenser eller udtryk I ikke forstår, er det altid en god idé at bruge Google. Det kan give svar på det meste.

# 1. Forberedelse

## Dependencies
Som altid begynder vi med at importere de nødvendige `libraries`. Udover de sædvanlige moduler skal vi også bruge `glob`. 

In [ ]:
import os                       # os tillader os bl.a. at finde filplaceringer på computeren
import numpy as np              # Numpy leverer noget af matematikken, der ligger under Pandas 
import pandas as pd             # Pandas tillader os at importere, oprette og manipulere data frames
import matplotlib.pyplot as plt # Importerer underbiblioteket pyplot fra pakken matplotlib
from nltk.text import Text      # nltk indholder mange forskellige funktioner, der kan bruges til tekstanalyse
import glob                     # glob-modulet hjælper med at finde specifikke filnavne

Herudover skal vi også importere `SpaCy`. Når vi bruger spacy, skal vi udover at importere modulet, også loade en sprogmodel. Der er tre modeller at vælge mellem: `da_core_news_sm`, `da_core_news_md`, `da_core_news_lg`, en lille (sm), en mellem (md) og en stor (lg). Størrelsen angiver, hvor stort et korpus modellen er blevet trænet på. 

**Importér** SpaCy og **load** den store danske sprog-model. Den kører lidt langsommere end de andre, men er til gengæld mere præcis. Skal man arbejde med meget store tekstmængder, kan det være en idé at bruge en mindre model. Det er en afvejning af forbrug af regnekraft og tid mod præcision.

In [ ]:
import spacy          
nlp = spacy.load("da_core_news_lg")

# 2. Import og klargøring af tekster
I denne notebook skal vi arbejde med de seneste 10 års nytårstaler. To af Mette Frederiksen, fire af Lars Løkke Rasmussen og fire af Helle Thorning Schmidt. 

**Placér** mappen med de downloadede `.txt`-filer i den mappe i den mappe, hvori i har gemt jeres script.

## Find den aktuelle sti
Hvis I ikke kender den aktuelle sti, kan nedenstående kommando bruges. Kommandoen forudsætter, at `os`-modulet er **importeret** (hvilket vi gjorde ovenfor):

`directory_path = os.getcwd()`<br>
`print(directory_path)`

Kommandoen `.getcwd()`returnerer den mappe I aktuelt arbejder i, ofte omtalt som *current working directory* (cwd).

I kan **copy/paste** den printede fil-sti, når vi skal om lidt skal importere talerne.

In [ ]:
directory_path = os.getcwd()
print(directory_path)

## Virker både på mac og pc
Hvis du i stedet arbejder på en mac, skal du bruge denne sekvens (pc-brugere kan også bruge denne sekvens). Med `glob`-modulet, som vi importerede indledningsvis, har vi mulighed for at matche filnavne. Asterisken i `\Taler\*.txt` betyder **alle filer** med endenlsen `.txt`. På denne måde åbner vi kun de tekst-filer, der ligger i mappen.

De problemer, vi tidligere er stødt på i forbindelse med afvikling af import-loop på mac-computere, skyldes, at Mac OS (styresystemet) gemmer skjulte filer i alle mapper. Filerne har praktiske funktioner og kan **ikke** ses i **pathfinder**. De skaber normalt ikke problemer, bortset fra når vi skal importere filer med Python. Ved at bruge `glob`-modulet bliver det muligt kun at importere de filer, der har endelsen `.txt`, hvilket sikrer, at de skjulte filer sorteres fra.

**Bemærk** at I enten kan vælge at copy/paste hele stien eller kan vælge `.\Taler\`, hvor punktummet angiver på på den aktuelle mappe (cwd).

**Husk** at vende skråstregerne i fil-stierne, hvis I arbejder på en mac.

In [ ]:
taler = []
#for roman in glob.glob(r'C:\Users\au100440\Desktop\1_Tæl din tekst\Jupyter_scripts\Taler\*.txt'): 
for roman in glob.glob(r'.\Taler\*.txt'): # henter alle filer i mappen 'Taler', der har fileendelsen .txt    
    with open(roman, "r", encoding="utf-8") as f: 
         taler.append(f.read().replace("\n"," ").replace("*"," "))

For at sikre os, at filerne er importeret, som de skal, vi printe de første tegn i hver tekst.

Med dette `for loop` beder vi om at få printet de første tyve tegn fra hvert element (teksterne er stadig en lang `string`) fra listen 'Taler'.

`for t in taler:`<br>
&emsp;`print(t[:20])`

In [ ]:
for t in taler:
    print(t[:30])

## Organisér listen i en bestemt rækkefølge
Hvis talerne ikke er blevet indlæst i den ønskede rækkefølge, kan I **reorganisere** elementerne på listen på følgende måde.

`nyRække = [5, 3, 2, 0, 1, 4, 6, 9, 8, 7]`<br>
`taler_reorg = [taler[t] for t in nyRække]`

Rækkefølgen i `nyRække` er et eksempel. Kig på listen ovenfor, og giv HTS_2012 nummer **1**, HTS_2013 nummer **2** osv. 

Den anden kodelinje laver en ny liste `taler_reorg`, hvor den som input tager tallene (i den rækkefølge de står) fra `nyRække` og bruger dem som indeks for udvælgelse af elementer fra listen `taler`. **Læses** noget i retning af 'Tag første tal fra `ny_række` (dvs 5); udvælg elementet med index 5 fra listen `taler`; tilføj elementet til listen `taler_reorg`. Gentag sekvense med andet tal fra `nyRække` (dvs 3) osv.'

**Print** den nye liste for at tjekke rækkefølgen. Brug samme loop som ovenfor.

Hvis I har reorganiseret talerne, så brug lsiten `taler_reorg` ellers den oprindelige liste `taler`. Nedenfor fortsætter jeg med `taler`, men I kan bare æbdre det til `taler_reorg`, eller hvad I nu kan finde på at kalde listen.

## Titler
Næste skridt er at lave en liste med titler, som vi kan bruge, når vi skal lave vores `data frame`.

Ved hjælp af `list comprehension` laver vi en liste, hvor vi splitter hver tale o ord, dvs. ved blanktegn, og derefter tilføjer det første element fra hver liste (index 0), dvs titlen, til en ny liste som vi kalder "titler":

`titler = [t.split(" ")[0] for t in taler]`

**Kør** kodesekvensen i feltet nedenfor. **Diskutér** hvad de enkelte dele betyder. **Inspicér** indholdet af variablen `titler` og tjek at alt ser ud som det skal.

In [ ]:
titler = [t.split(" ")[0] for t in taler]

## Tekst uden titel
Herefter fjerner vi tilen fra alle taler, så vi får en ren tekst. Da alle taler begynder med 'Godaften' gøres dette lette ved at splitte teksten ved 'godaften' og derefter tilføje anden del (indeks 1) til listen `taler_ren`.

`taler_ren = [t.split("Godaften")[1] for t in taler]`

**Kør** kodesekvensen i feltet nedenfor. **Diskutér** hvad de enkelte dele betyder. **Inspicér** indholdet af variablen `taler_ren` og tjek at alt ser ud som det skal.

In [ ]:
taler_ren = [t.split("Godaften")[1] for t in taler]

In [ ]:
talerLowSplit = [t.lower().split() for t in taler_ren] 

# 3. Lav en `data frame`
Vi kan nu lave en `data frame` ud af kapitlerne på listen med følgende kommando:

`df_nytår = pd.DataFrame({"ID": titler, "Tekst": taler_ren})`

**Indtast** koden i feltet nedenfor. **Læs** kodesekvensen og **diskutér** hvad de forskellige dele betyder. **Inspicér** jeress dataframe og tjek, at alt er i orden. 

In [ ]:
df_nytår = pd.DataFrame({"ID": titler, "Tekst": taler_ren, "bagLow": talerLowSplit})

In [ ]:
df_nytår.head()

In [ ]:
def dansk(txt):
    counter = 0
    for w in txt:
        if "dansk" in w:
            counter += 1
    return counter

def danm(txt):
    counter = 0
    for w in txt:
        if "danm" in w:
            counter += 1
    return counter

def interna(txt):
    counter = 0
    for w in txt:
        if "interna" in w:
            counter += 1
    return counter

def eu(txt):
    counter = 0
    for w in txt:
        if "eu" in w:
            counter += 1
    return counter

def verden(txt):
    counter = 0
    for w in txt:
        if "verden" in w:
            counter += 1
    return counter

In [ ]:
df_nytår["dansk"] = df_nytår.bagLow.apply(dansk)
df_nytår["danm"] = df_nytår.bagLow.apply(danm)
df_nytår["interna"] = df_nytår.bagLow.apply(interna)
df_nytår["eu"] = df_nytår.bagLow.apply(eu)
df_nytår["verden"] = df_nytår.bagLow.apply(verden)


df_nytår["National"] = df_nytår.dansk + df_nytår.danm
df_nytår["International"] = df_nytår.eu + df_nytår.interna + df_nytår.verden
df_nytår

# 4. SpaCy

Vi har allerede importeret `SpaCy`, load'et den store danske sprogmodel og gemt den unde navnet `nlp`, så SpaCy-modulet er klar til brug.

## Tilføj kolonne med nlp-objekter
Med `.apply()` kan vi anvende `nlp()` på alle teksterne i vores `data frame`. Vi gemmer i samme omgang nlp-objekterne i en ny kolonne med navnet 'nlp_Tekst'.

`df_nytår["nlp_Tekst"] = df_nytår.Tekst.apply(nlp)`

**Indtast** koden i feltet nedenfor. **Læs** kodesekvensen og **diskutér** hvad de forskellige dele betyder. 

**Inspicér** jeress dataframe og tjek, at alt er i orden. 

In [ ]:
df_nytår["nlp_Tekst"] = df_nytår.Tekst.apply(nlp)

# 5. Sentiment analyse med `afinn`
**Sentiment analyse** dækker over forskellige teknikker til at beskrive om en tekst er udtryk for **positive** eller **negative** holdninger. Dette kan gøres på forskellige måder. Med `afinn`-modulet, der også virker på dansk, tildeles udvalgte ord, typisk navneord og adjektiver, en score mellem 5 (mest positiv) og -5 (mest negativ). Tallene lægges sammen til en samlet score for teksten.

For at kunne bruge `afinn`-modulet, skal i åbne terminalen og indtaste følgende kommando efterfulgt af Enter:

`pip install afinn`

Mere om sentiment analyse:
https://www.freecodecamp.org/news/what-is-sentiment-analysis-a-complete-guide-to-for-beginners/


## Definér funktion
Som udgangspunkt kan vi godt køre en sentiment analyse på hele teksten. Vi kan dog også vælge at lave analyse på udvalgte ordklasser - dette kan varieres på forskellige måder afhængigt af formålet.

Nedenfor **defineres** en funktion, der hver tekst udtrækker en liste af **adjektiver** og **substantiver**. Dette gøres ved hjælp af `.pos_`-kommandoen fra SpaCy-modluet.

Funktionen kan let modificeres, ved at tilføje eller fjerne tags.

**Læs** kodesekvensen og **diskutér** hvad de forskellige dele betyder. 

In [ ]:
def find_ord(txt):
    x = []
    y = list(txt.sents)
    for s in y:                   
        for w in s:               
            if w.pos_ == "ADJ" or w.pos_ == "NOUN":
                x.append(w.text)
    return sorted(set(x))        

In [ ]:
def find_sted(txt):
    x = []
    y = list(txt.ents)
    for e in y:               
        if e.label_ == "LOC":
            x.append(e.text)
    return x

Med `.apply()` kan vi let tilføje en ny kolonne med de nye ordlister.

`df_nytår["Subst_adj"] = df_nytår.nlp_Tekst.apply(find_ord)`

**Indtast** koden i feltet nedenfor. **Læs** kodesekvensen og **diskutér** hvad de forskellige dele betyder. 

In [ ]:
df_nytår["Subst_adj"] = df_nytår.nlp_Tekst.apply(find_ord)

In [ ]:
df_nytår["GPE"] = df_nytår.nlp_Tekst.apply(find_sted)

In [ ]:
df_nytår

In [ ]:
locs = []
for i in range(0,10):
    locs.append(df_nytår.loc[i,'GPE'])
locs


In [ ]:
for i in range(0,9):
    print(i)

`afinn`-modulet tager strings som input, så inden vi kan lave analysen, skal vi have transformeret ordlisten til en samlet streng. Dette gøres med `.join()`-kommandoen.

Da vi skal gøre det for alle ti taler, er det nemmest at lave en funktion. `" "` angiver, at der skal indsættes et blanktegn mellem hvert ord, der joines - altså den modsatte proces at `split`.

In [ ]:
def streng(lst):
    return " ".join(lst)

Vi kan nu anvende funktionen på kolonne med ordlisterne:

In [ ]:
df_nytår["Subst_adj_streng"] = df_nytår.Subst_adj.apply(streng)
df_nytår.head(3)

Herefter **importerer** vi `afinn`-modulet og loader den danske ordliste. Fremgangsmåden minder om den måder vi arbejder med SpaCy.

In [ ]:
from afinn import Afinn
afinn = Afinn(language='da')

Vi kan nu til føje en ny kolonne med en sentiment score:

In [ ]:
df_nytår["Sentiment_score"] = df_nytår.Subst_adj_streng.apply(afinn.score)
df_nytår

In [ ]:
df_newYear = df_nytår[['ID','National', 'International','Sentiment_score']].copy()
df_newYear = df_newYear.set_index('ID')
df_newYear.head(3)

In [ ]:
df_newYear.plot(kind="bar",figsize=(25,10), color=['orangered', 'dodgerblue', 'limegreen'])

plt.title("Nytårstalernes sentiment score")
plt.ylabel("Sentiment score")
plt.xlabel("Nytårstale")

plt.legend()
plt.show()

Hvis vi vil se nærmere på, hvordan scoren er sammensat, kan vi score de enkelte ord i en tale.

**Læs** kodesekvensen og **diskutér** hvad de forskellige dele betyder. 

In [ ]:
for w in taler[5].split(" "):
    print(w, afinn.score(w))

In [ ]:
from wordcloud import WordCloud

In [ ]:
locsStr = []
for l in locs:
    x = " ".join(l)
    locsStr.append(x)
locsStr

In [ ]:
#for i in range(0,10):
wc0 = WordCloud(width = 1200, height = 800,
        background_color ='white',
        max_words=20,
        min_font_size = 10).generate(locsStr[0])
plt.imshow(wc0, interpolation='bilinear')



In [ ]:
wc1 = WordCloud(width = 1200, height = 800,
        background_color ='white',
        max_words=20,
        min_font_size = 10).generate(locsStr[1])
plt.imshow(wc1, interpolation='bilinear')



In [ ]:
wc2 = WordCloud(width = 1200, height = 800,
        background_color ='white',
        max_words=20,
        min_font_size = 10).generate(locsStr[2])
plt.imshow(wc2, interpolation='bilinear')



In [ ]:
wc3 = WordCloud(width = 1200, height = 800,
        background_color ='white',
        max_words=20,
        min_font_size = 10).generate(locsStr[3])
plt.imshow(wc3, interpolation='bilinear')



In [ ]:
wc4 = WordCloud(width = 1200, height = 800,
        background_color ='white',
        max_words=20,
        min_font_size = 10).generate(locsStr[4])
plt.imshow(wc4, interpolation='bilinear')



In [ ]:
wc5 = WordCloud(width = 1200, height = 800,
        background_color ='white',
        max_words=20,
        min_font_size = 10).generate(locsStr[5])
plt.imshow(wc5, interpolation='bilinear')



In [ ]:
wc6 = WordCloud(width = 1200, height = 800,
        background_color ='white',
        max_words=20,
        min_font_size = 10).generate(locsStr[6])
plt.imshow(wc6, interpolation='bilinear')



In [ ]:
wc7 = WordCloud(width = 1200, height = 800,
        background_color ='white',
        max_words=20,
        min_font_size = 10).generate(locsStr[7])
plt.imshow(wc7, interpolation='bilinear')



In [ ]:
wc8 = WordCloud(width = 1200, height = 800,
        background_color ='white',
        max_words=20,
        min_font_size = 10).generate(locsStr[8])
plt.imshow(wc8, interpolation='bilinear')



In [ ]:
wc9 = WordCloud(width = 1200, height = 800,
        background_color ='white',
        max_words=20,
        min_font_size = 10).generate(locsStr[9])
plt.imshow(wc9, interpolation='bilinear')

En anden måde at får overblik er ved at bruge funktionen `counter()`, der kan tælle antallet af forekomster af de enkelte ord.

**Læs** kodesekvensen og **diskutér** hvad de forskellige dele betyder. 


In [ ]:
from collections import Counter
for l in locs:
    print("Optælling", Counter(l), "\n")

## Plot

Vi kan plotte vores sentiment score som et stolpediagram.

In [ ]:
plt.figure(figsize=(25,10))

plt.title("Nytårstalernes sentiment score")

width = 0.40
Pos = np.arrange(10)
plt.bar(Pos - 0.3, df_nytår.ID, df_nytår.Sentiment_score, width,label="Sentiment")
plt.bar(Pos,df_nytår.ID,df_nytår.National, width, label="Danmark")
plt.bar( Pos = +0.3, df_nytår.ID, df_nytår.International,width, label="Verden")
plt.ylabel("Sentiment score")
plt.xlabel("Nytårstale")

plt.legend()
plt.savefig("nytårs_score.pdf", bbox_inches = "tight", dpi=200)
plt.show()

# 6. Gem data frame som csv-fil
For at gemme data-framen som csv-fil skal i bruge følgende kommando. Filnavnet er naturligvis valgfrit:

`df_nytår.to_csv('nytårstaler_sentiment.csv', index=False)`


Tilføjelsen `index=False` sikrer, at data-framen ikke gemmes med det automatisk generede index (første kolonne: 0, 1, 2, osv.). Der genereres som default et nyt index, hver gang data-framen åbnes, og vi ville derfor have først to, så tre indekser osv., hvis vi gemmer indekse hver gang.

Filen gemmes som default i den fil-mappe, hvori scriptet er gemt.

**Gem**  data-frame.